# DecisionTree 과제

#### 개요

과제 1번\
본인이 구현한 함수를 통해 다음 문제를 풀어주세요!

1) 변수 ‘income’의 이진분류 결과를 보여주세요.\
2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.\
3) 문제 2에서 제시한 feature로 DataFrame을 split한 후, 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요

과제 2번\
ID3 알고리즘을 통해 최초 split feature가 무엇인지 도출하는 과정을 계산해주세요!

#### 데이터 출처
https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv

# Data Loading

In [1]:
import pandas as pd 
import numpy as np
import math

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

In [3]:
def get_gini(df, label):
    # label열 안에서 unique값들과 이들의 개수를 구합니다
    unique_counts = df[label].value_counts()
    # unique의 행 개수 = label 클래스 개수 
    gini = 1
    # Pj 구하기
    length = len(df)
    for i in unique_counts:
        # P(yes) = yes의 count / df의 길이
        p = i/length
        # 1로 초기화한 gini에 계속 클래스 개수만큼 Pj를 구해서 빼주었습니다
        gini -= pow(p,2)
    return gini    

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기

In [5]:
def get_binary_split(df, attribute):
    def get_combination(li1, li2, result):
        # li1은 조합을 구할 때 사용하는 리스트, li2는 조합을 구할 리스트, li는 결과를 담을 리스트 
        for i in range(len(li2)):
            # 1) li1 복사 후 i 대로 li2에서 하나씩 추가하고 result에 담음
            # 3) 재귀로 다시 불러졌을 때 그대로 new_li1을 복사 
            new_li1 = li1[:]
            new_li1.append(li2[i])
            result.append(new_li1)
            # 2) li2의 범위는 하나씩 좁아짐
            new_li2 = li2[i+1:]
#             print(new_li1, new_li2)
            # 재귀로 계속 돌림
            get_combination(new_li1, new_li2, result)
    
    # 함수에 필요한 리스트들
    result = []
    li = []
    # 특정 attribute의 유니크 값들 
    unique = list(df[attribute].unique())
    # 조합 구하는 함수
    get_combination(li, unique, result)
    # 유니크 리스트 자체인 자기 자신값은 필요없음 
    result.remove(unique)
    
    return result

get_binary_split(pd_data,"age")

[['youth'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged'],
 ['middle_aged', 'senior'],
 ['senior']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기

In [6]:
def get_attribute_gini_index(df, attribute, label): 
    # 결과가 담길 dictionary
    dictionary = {}
    
    # 조합들을 반복문으로 각각의 지니값을 구함
    for classes in get_binary_split(df,attribute):
        # 첫 번째 Dataframe, 그냥 classes 그 자체인 데이터
        df1 = pd.DataFrame(columns=df.columns)
        # split한 클래스끼리 묶음
        for class_ in classes:
            # class_ 조건에 따라 df에서 끌어옴
            temp = df.loc[df[attribute] == class_]
            # DataFrame으로 만들기
            temp_df = pd.DataFrame(temp, columns=df.columns)
            # df1에 concat
            df1 = pd.concat([df1,temp_df])
        # 반복문이 끝나면 클래스의 조합에 따라 concat된 df1 완성
        
        # 대응되는 두 번째 Dataframe ex) df1 = youth, df2  = mid, senior
        # 전체 데이터인 pd_data에서 df1를 뺀 나머지를 df2로 만들려고 해요
        # df1 index값들을 뽑아오고
        df_index = df1.index.values
        # boolean 값으로 만들고 
        bool_li = df.index.isin(df_index) # df와 원래 pd_data 비교
        # 전체 데이터에서 df1이 아닌 데이터만 골라 냄
        df2 = df[~bool_li]

        # 각각의 df의 지니값 구하기
        gini1 = (len(df1) / len(df)) * get_gini(df1,label)
        gini2 = (len(df2) / len(df)) * get_gini(df2,label)
        # 최종 df
        gini = gini1 + gini2
        # 딕셔너리에 정리하기
        key = '_'.join(classes)
        dictionary[key] = gini
        
    return dictionary

In [7]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'youth_middle_aged': 0.4571428571428572,
 'youth_senior': 0.35714285714285715,
 'middle_aged': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937,
 'senior': 0.4571428571428572}

In [8]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

## 과제 2

In [9]:
print(len(pd_data))
pd_data.head()

14


,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes


In [10]:
p = pd_data['income'].value_counts()
p

medium    6
high      4
low       4
Name: income, dtype: int64

In [11]:
#Info(D) 
def getInfoD(df, label):
    p = df[label].value_counts()
    classes = df[label].unique()
    score = 0
    for i in classes:
        score += entropy(p.loc[i] / float(len(df)))
    return score
def entropy(p):
     return -p*math.log2(p)

getInfoD(pd_data,'class_buys_computer')

0.9402859586706311

In [12]:
# InfoA(Di)
# high=1 , medium=2 , low=3
# income과 buy 조건 비교
'''
income = df['income'].unique()
buy = df['class_buys_computer'].unique()
income_p = df['income'].value_counts()
for i in income:
    for j in buy:
        if i in dic:
            dic[i] += entropy(check(i,j,pd_data) / income_p.loc[i])
        else:
            dic[i] = entropy(check(i,j,pd_data) / income_p.loc[i])
        
print(dic)
        
label1 = income_p['high'] / len(df) * dic['high']
label2 = income_p['medium'] / len(df) * dic['medium']
label3 = income_p['low'] / len(df) * dic['low']

info_income = label1 + label2 + label3
print('info_income: ',info_income)

gain_income = info_D - info_income
print('gain: ',gain_income)
'''


# 함수로 만든 것들
def getInfoA(df, attribute, label):
    att = df[attribute].unique()
    lab = df[label].unique()
    att_p = df[attribute].value_counts()
    
    dic = {}
    for i in att:
        for j in lab:
            if i in dic:
                dic[i] += entropy(checkP(attribute, i, label, j, df) / float(att_p.loc[i]))
            else:
                dic[i] = entropy(checkP(attribute, i, label, j, df) / float(att_p.loc[i]))
                
    score = 0
    for i in att:
        score += att_p[i] / len(df) * dic[i]
    return score

def checkP(attribute, att_val, label, label_val, df):
    df_ = df.loc[(df[attribute] == att_val) & (df[label] == label_val)]
    return len(df_)

##문제2 답안
def getBestGain(df, att_li, label):
    info_D = getInfoD(df,label)
    dic = {}
    for i in att_li:
        info_A = getInfoA(df, i, label)
        dic[i] = info_D - info_A 
    score = max(dic.items())
    return score
        
att_li = list(pd_data.columns.values)
att_li.pop()
# age에서 log2할때 0을 넣어주게 되어서 오류가 걸립니다. 아직 해결 못했어요....
att_li.pop(0)
print(getBestGain(pd_data, att_li, 'class_buys_computer'))

('student', 0.15183550136234159)


* ID3 알고리즘으로 student가 최초 split feature로 선정되었습니다.